# W2 - Accessibility (Distance Fields)

In this workshop we will learn the foundations to quantitatively approach spatial accessibility. We will learn about distance fields, construct a euclidean distance field, and construct a manifold distance field.

## 0. Initialization

### 0.1 Importing the packages

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np

pv.set_jupyter_backend("ipyvtklink")

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

### 0.2 import meshes

In [2]:
envelope_path = os.path.relpath('../data/compulsory_envelope.obj')
context_path = os.path.relpath('../data/immediate_context.obj')

# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)
print(context_mesh.is_watertight)

True
False


In [3]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
# p.show()

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x55ff821c3790) at 0x7feb89b2da00>

### 0.3 Importing the Envelope Lattice

In [4]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

In [5]:
# initiating the plotter
p = pv.Plotter()

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
# p.show()

/home/tandao/.conda/envs/scds/lib/python3.8/site-packages/pyvista/core/dataset.py:1332: PyvistaDeprecationWarning: Use of `cell_arrays` is deprecated. Use `cell_data` instead.
  warnings.warn(


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x55ff80973df0) at 0x7feb895b4e80>

### 0.4 Importing the Street Points

In [6]:
# import the streetnetwork as a point cloud
street_pc = tg.cloud_from_csv("../data/main_street_points.csv")

In [7]:
# initiating the plotter
p = pv.Plotter()

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# fast visualization of the point cloud
street_pc.fast_notebook_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
# p.show()

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x55ff82b773b0) at 0x7feb894f9ee0>

## 1. Euclidean Distance Lattice

### 1.1 Distance Matrix

In [8]:
# extracting the centroid of all voxels
env_cens = envelope_lattice.centroids_threshold(-1)

# initializing the distance matrix
dist_m = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_v = []
    # for each street point ...
    for street_point in street_pc:
        # find the difference vector
        diff = voxel_cen - street_point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_v.append(dist)
    # add the distance vector to the distance matrix
    dist_m.append(dist_v)
# change the distance matrix type, from list to array
dist_m = np.array(dist_m)


"""
st_m_shape = (env_cens.shape[0], street_pc.shape[0], 3)
st_m = np.broadcast_to(street_pc, st_m_shape)
ep_m_shape = (street_pc.shape[0], env_cens.shape[0], 3)
ep_m = np.broadcast_to(env_cens, ep_m_shape).transpose((1,0,2))
dist_m = np.linalg.norm(st_m - ep_m, axis=2)
"""

'\nst_m_shape = (env_cens.shape[0], street_pc.shape[0], 3)\nst_m = np.broadcast_to(street_pc, st_m_shape)\nep_m_shape = (street_pc.shape[0], env_cens.shape[0], 3)\nep_m = np.broadcast_to(env_cens, ep_m_shape).transpose((1,0,2))\ndist_m = np.linalg.norm(st_m - ep_m, axis=2)\n'

### 1.2 Distance to Closest Street Point

In [9]:
# find the distance to the closest street point for each voxel
min_dist = dist_m.min(axis=1)
# convert the minimum distance list to a lattice
street_eu_distance_lattice = tg.to_lattice(min_dist.reshape(envelope_lattice.shape), envelope_lattice)
# zero the value of the exterior voxels
envelope_eu_dist_lattice = street_eu_distance_lattice * envelope_lattice

In [10]:
# initiating the plotter
p = pv.Plotter()

l = envelope_eu_dist_lattice * envelope_lattice

# remapping
l = 250 * (l - l.min()) / l.max()

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = l.shape
# The bottom left corner of the data set
grid.origin = l.minbound
# These are the cell sizes along each axis
grid.spacing = l.unit

# Add the data values to the cell data
grid.point_data["Distance"] = l.flatten(order="F")  # Flatten the Lattice

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# fast visualization of the point cloud
street_pc.fast_notebook_vis(p)

# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6]) * 1.5
p.add_volume(grid, cmap="coolwarm", opacity=opacity, shade=True, show_scalar_bar=False)

# plotting
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 2 Manifold Distance Lattice

### 2.1 Selecting the Closest Voxels

In [11]:
# selecting the closest voxels by setting a threshold 
street_connection_lattice = (0 < envelope_eu_dist_lattice) * (envelope_eu_dist_lattice < 12)

### 2.2. The Stencil

In [12]:
# creating neighborhood definition
stencil = tg.create_stencil("von_neumann", 1, 1)
# setting the center to zero
stencil.set_index([0,0,0], 0)

print(stencil)

[[[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 1 0]
  [1 0 1]
  [0 1 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]]


### 2.3 Initializing the Manifold Distance Lattice

In [13]:
# retrieve the neighbour list of each cell
neighs = street_connection_lattice.find_neighbours(stencil)

# set the maximum distance to sum of the size of the lattice in all dimensions.
max_dist = np.sum(street_connection_lattice.shape)

# initialize the street network distance lattice with all the street cells as 0, and all other cells as maximum distance possible
mn_dist_lattice = 1 - street_connection_lattice
mn_dist_lattice[mn_dist_lattice==1] = max_dist

# flatten the distance lattice for easy access
mn_dist_lattice_flat = mn_dist_lattice.flatten()

# flatten the envelope lattice
env_lat_flat = envelope_lattice.flatten()

### 2.4 Breadth-First Traversal

In [14]:
# main loop for breath-first traversal
for i in range(1, max_dist):
    # find the neighbours of the previous step
    next_step = neighs[mn_dist_lattice_flat == i - 1]
    # find the unique neighbours
    next_unq_step = np.unique(next_step.flatten())
    # check if the neighbours of the next step are inside the envelope
    validity_condition = env_lat_flat[next_unq_step]
    # select the valid neighbours
    next_valid_step = next_unq_step[validity_condition]
    # make a copy of the lattice to prevent overwriting in the memory
    mn_nex_dist_lattice_flat = np.copy(mn_dist_lattice_flat)
    # set the next step cells to the current distance
    mn_nex_dist_lattice_flat[next_valid_step] = i
    # find the minimum of the current distance and previous distances to avoid overwriting previous steps
    mn_dist_lattice_flat = np.minimum(mn_dist_lattice_flat, mn_nex_dist_lattice_flat)
    
    # check how many of the cells have not been traversed yet
    filled_check = mn_dist_lattice_flat * env_lat_flat == max_dist
    # if all the cells have been traversed, break the loop
    if filled_check.sum() == 0:
        print(i)
        break

# reshape and construct a lattice from the street network distance list
mn_dist_lattice = mn_dist_lattice_flat.reshape(mn_dist_lattice.shape)

20


In [15]:

# the commented code here is the equivalent of the same process but without the assumption that we are working on a regular grid. Producing the distance lattice through this algorithm will take several hours. (Q+) Can you show why these two approaches are equivalent? Can you explain why the second algorithm takes more time? 
"""
# find the number of all voxels
vox_count = street_connection_lattice.size 

# initialize the adjacency matrix
adj_mtrx = np.zeros((vox_count,vox_count))

# Finding the index of the available voxels in avail_lattice
avail_index = np.array(np.where(street_connection_lattice == 1)).T

# fill the adjacency matrix using the list of all neighbours
for vox_loc in avail_index:
    # find the 1D id
    vox_id = np.ravel_multi_index(vox_loc, street_connection_lattice.shape)
    # retrieve the list of neighbours of the voxel based on the stencil
    vox_neighs = street_connection_lattice.find_neighbours_masked(stencil, loc = vox_loc)
    # iterating over the neighbours
    for neigh in vox_neighs:
        # setting the entry to one
        adj_mtrx[vox_id, neigh] = 1.0

# construct the graph 
g = nx.from_numpy_array(adj_mtrx)

# compute the distance of all voxels to all voxels using floyd warshal algorithm
dist_mtrx = nx.floyd_warshall_numpy(g)
"""

'\n# find the number of all voxels\nvox_count = street_connection_lattice.size \n\n# initialize the adjacency matrix\nadj_mtrx = np.zeros((vox_count,vox_count))\n\n# Finding the index of the available voxels in avail_lattice\navail_index = np.array(np.where(street_connection_lattice == 1)).T\n\n# fill the adjacency matrix using the list of all neighbours\nfor vox_loc in avail_index:\n    # find the 1D id\n    vox_id = np.ravel_multi_index(vox_loc, street_connection_lattice.shape)\n    # retrieve the list of neighbours of the voxel based on the stencil\n    vox_neighs = street_connection_lattice.find_neighbours_masked(stencil, loc = vox_loc)\n    # iterating over the neighbours\n    for neigh in vox_neighs:\n        # setting the entry to one\n        adj_mtrx[vox_id, neigh] = 1.0\n\n# construct the graph \ng = nx.from_numpy_array(adj_mtrx)\n\n# compute the distance of all voxels to all voxels using floyd warshal algorithm\ndist_mtrx = nx.floyd_warshall_numpy(g)\n'

In [16]:
# set the lattice to be visualized
l = mn_dist_lattice * envelope_lattice
# remapping
l = 250 * (l - l.min()) / l.max()

# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = l.shape
# The bottom left corner of the data set
grid.origin = l.minbound
# These are the cell sizes along each axis
grid.spacing = l.unit

# Add the data values to the cell data
grid.point_arrays["Street Access"] = l.flatten(order="F")  # Flatten the Lattice
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6]) * 1.5
p.add_volume(grid, cmap="coolwarm", opacity=opacity, shade=True, show_scalar_bar=False)

# plotting
p.show()

/home/tandao/.conda/envs/scds/lib/python3.8/site-packages/pyvista/core/dataset.py:1192: PyvistaDeprecationWarning: Use of `point_arrays` is deprecated. Use `point_data` instead.
  warnings.warn(


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Credits

In [17]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/earthy_workshops"
__summary__ = "Earthy Design Studio"